<a href="https://colab.research.google.com/github/pidipidi/samsung_planning/blob/main/notebooks/N02_robomimic_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Robomimic Get Started Tutorial

This notebook implements a simple training loop without the extensive features offered in robomimic such as logging and hyperparameter sweeping. Please refer to the [repository](https://github.com/ARISE-Initiative/robomimic) and the [documentation](https://robomimic.github.io/docs/introduction/overview.html) for the full set of features and the rest of the pipeline.

This notebook includes the following tutorials:

1. Set up robomimic development environment
2. Downloading task-specific dataset
3. Load a naive behavior cloning policy
4. Setup a simple training loop
5. Run policy training
6. Visualize the trained policy

###0. Use GPU to accelerate training

To use GPU runtime, click runtime on the top navigation part -> change runtime type -> select GPU as your accelerator

### 1. Set up development environment

The main dependencies of robomimic are
- torch
- numpy
- h5py
- robosuite
- mujoco
- tensorbordX
- egl_probe
- matplotlib


The full list is included in the requirements.txt file in the repo.

In [1]:
%env MUJOCO_GL=osmesa

WS_DIR = "/content/"
%cd $WS_DIR

# Clone the repo and install the basic requirements
!git clone --branch v0.4 https://github.com/ARISE-Initiative/robomimic
!pip install -e robomimic/ > /dev/null

import sys
import os
sys.path.append('./robomimic/')

env: MUJOCO_GL=osmesa
/content
Cloning into 'robomimic'...
remote: Enumerating objects: 3546, done.
remote: Counting objects: 100% (1660/1660), done.
remote: Compressing objects: 100% (380/380), done.
remote: Total 3546 (delta 1450), reused 1280 (delta 1280), pack-reused 1886 (from 2)
Receiving objects: 100% (3546/3546), 62.06 MiB | 14.03 MiB/s, done.
Resolving deltas: 100% (2467/2467), done.
Updating files: 100% (185/185), done.


**WARNING**: To exactly reproduce the setup from our study paper, mujoco and robosuite should be installed from source following the [instruction](https://robomimic.github.io/docs/introduction/installation.html#install-simulators) However, if you just want to train on our dataset, you could proceed with the
following default setup.

In [2]:
# install all system dependencies for mujoco-py
!sudo DEBIAN_FRONTEND=noninteractive apt install curl git libgl1-mesa-dev libgl1-mesa-glx libglew-dev \
         libosmesa6-dev software-properties-common net-tools unzip vim \
         virtualenv wget xpra xserver-xorg-dev libglfw3-dev patchelf > /dev/null

#install mujoco-py
!pip install mujoco > /dev/null

#install robosuite
!pip install robosuite > /dev/null



Extracting templates from packages: 100%


## 2. Download demonstration dataset for a task

For robomimic tasks, we organize the demonstration datasets by
- task name (e.g., lift)
- data source (ph - proficient human, mh - multi human, mg - machine-generated)
- observation type (low_dim or image)

For more details of the dataset structure, visit [robomimic documentation](https://robomimic.github.io/docs/datasets/robomimic_v0.1.html) and the [dataset tutorial](https://github.com/ARISE-Initiative/robomimic/blob/master/examples/notebooks/datasets.ipynb)


Here we demonstrate downloading the proficient human (`ph`) dataset with low-dimensional (`low_dim`) observation for the `lift` task.



In [3]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils

# the dataset registry can be found at robomimic/__init__.py
from robomimic import DATASET_REGISTRY, HF_REPO_ID

# set download folder and make it
download_folder = WS_DIR + "robomimic_data/"
os.makedirs(download_folder, exist_ok=True)

# download the dataset
task = "lift"
dataset_type = "ph"
hdf5_type = "low_dim"
FileUtils.download_file_from_hf(
    repo_id=HF_REPO_ID,
    filename=DATASET_REGISTRY[task][dataset_type][hdf5_type]["url"],
    download_dir=download_folder,
)

# enforce that the dataset exists
dataset_path = os.path.join(download_folder, "low_dim_v15.hdf5")
assert os.path.exists(dataset_path)

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /content/./robomimic/robomimic/scripts/setup_macros.py
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


low_dim_v15.hdf5:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

In [4]:
# Here we can print out the data metadata
!python robomimic/robomimic/scripts/get_dataset_info.py --dataset $dataset_path


total transitions: 9666
total trajectories: 200
traj length mean: 48.33
traj length std: 6.116461395284041
traj length min: 36
traj length max: 64
action min: -1.0
action max: 1.0

==== Filter Keys ====
filter key 20_percent with 40 demos
filter key 20_percent_train with 36 demos
filter key 20_percent_valid with 4 demos
filter key 50_percent with 100 demos
filter key 50_percent_train with 90 demos
filter key 50_percent_valid with 10 demos
filter key train with 180 demos
filter key valid with 20 demos

==== Env Meta ====
{
    "env_name": "Lift",
    "env_version": "1.5.1",
    "type": 1,
    "env_kwargs": {
        "has_renderer": false,
        "has_offscreen_renderer": false,
        "ignore_done": true,
        "use_object_obs": true,
        "use_camera_obs": false,
        "control_freq": 20,
        "controller_configs": {
            "type": "BASIC",
            "body_parts": {
                "right": {
                    "type": "OSC_POSE",
                    "input_max": 1

## 3. Build a simple behavior cloning model

Follows the default hyperparameter in `robomimic/config/bc_config.py`.

In [5]:
# import all utility functions

import numpy as np

import torch
from torch.utils.data import DataLoader

import robomimic
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.test_utils as TestUtils
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.train_utils as TrainUtils
from robomimic.utils.dataset import SequenceDataset

from robomimic.config import config_factory
from robomimic.algo import algo_factory

In [6]:
def get_example_model(dataset_path, device):
    """
    Use a default config to construct a BC model.
    """

    # default BC config
    config = config_factory(algo_name="bc")

    # read config to set up metadata for observation modalities (e.g. detecting rgb observations)
    ObsUtils.initialize_obs_utils_with_config(config)

    # read dataset to get some metadata for constructing model
    # all_obs_keys determines what observations we will feed to the policy
    shape_meta = FileUtils.get_shape_metadata_from_dataset(
        dataset_path=dataset_path,
        all_obs_keys=sorted((
            "robot0_eef_pos",  # robot end effector position
            "robot0_eef_quat",   # robot end effector rotation (in quaternion)
            "robot0_gripper_qpos",   # parallel gripper joint position
            "object",  # object information
        )),
    )

    # make BC model
    model = algo_factory(
        algo_name=config.algo_name,
        config=config,
        obs_key_shapes=shape_meta["all_shapes"],
        ac_dim=shape_meta["ac_dim"],
        device=device,
    )
    return model

In [7]:
device = TorchUtils.get_torch_device(try_to_use_cuda=True)
model = get_example_model(dataset_path, device=device)

print(model)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['object', 'robot0_eef_pos', 'robot0_gripper_qpos', 'robot0_eef_quat']
using obs modality: rgb with keys: []
using obs modality: depth with keys: []
using obs modality: scan with keys: []
ObservationKeyToModalityDict: action not found, adding action to mapping with assumed low_dim modality!
BC (
  ModuleDict(
    (policy): ActorNetwork(
        action_dim=7
  
        encoder=ObservationGroupEncoder(
            group=obs
            ObservationEncoder(
                Key(
                    name=object
                    shape=[10]
                    modality=low_dim
                    randomizer=None
                    net=None
                    sharing_from=None
                )
                Key(
                    name=robot0_eef_pos
                    shape=[3]
                    modality=low_dim
                    randomizer=None
                    net

## 4. Build a simple training loop

Here we build a simple data loader pipeline and a training loop. Note that this code snippet is only instructional and is a stripped-down version of robomimic's main training loop (`robomimic/scripts/train.py`).

In [8]:
"""
WARNING: This code snippet is only for instructive purposes, and is missing several useful
         components used during training such as logging and rollout evaluation.
"""
def get_data_loader(dataset_path):
    """
    Get a data loader to sample batches of data.
    Args:
        dataset_path (str): path to the dataset hdf5
    """
    dataset = SequenceDataset(
        hdf5_path=dataset_path,
        obs_keys=(                      # observations we want to appear in batches
            "robot0_eef_pos",
            "robot0_eef_quat",
            "robot0_gripper_qpos",
            "object",
        ),
        dataset_keys=(                  # can optionally specify more keys here if they should appear in batches
            "actions",
            "rewards",
            "dones",
        ),
        load_next_obs=True,
        frame_stack=1,
        seq_length=10,                  # length-10 temporal sequences
        pad_frame_stack=True,
        pad_seq_length=True,            # pad last obs per trajectory to ensure all sequences are sampled
        get_pad_mask=False,
        goal_mode=None,
        hdf5_cache_mode="all",          # cache dataset in memory to avoid repeated file i/o
        hdf5_use_swmr=True,
        hdf5_normalize_obs=False,
        filter_by_attribute=None,       # can optionally provide a filter key here
    )
    print("\n============= Created Dataset =============")
    print(dataset)
    print("")

    data_loader = DataLoader(
        dataset=dataset,
        sampler=None,       # no custom sampling logic (uniform sampling)
        batch_size=100,     # batches of size 100
        shuffle=True,
        num_workers=0,
        drop_last=True      # don't provide last batch in dataset pass if it's less than 100 in size
    )
    return data_loader


def run_train_loop(model, data_loader, num_epochs=50, gradient_steps_per_epoch=100):
    """
    Note: this is a stripped down version of @TrainUtils.run_epoch and the train loop
    in the train function in train.py. Logging and evaluation rollouts were removed.
    Args:
        model (Algo instance): instance of Algo class to use for training
        data_loader (torch.utils.data.DataLoader instance): torch DataLoader for
            sampling batches
    """
    # ensure model is in train mode
    model.set_train()

    for epoch in range(1, num_epochs + 1): # epoch numbers start at 1

        # iterator for data_loader - it yields batches
        data_loader_iter = iter(data_loader)

        # record losses
        losses = []

        for _ in range(gradient_steps_per_epoch):

            # load next batch from data loader
            try:
                batch = next(data_loader_iter)
            except StopIteration:
                # data loader ran out of batches - reset and yield first batch
                data_loader_iter = iter(data_loader)
                batch = next(data_loader_iter)

            # process batch for training
            input_batch = model.process_batch_for_training(batch)

            # forward and backward pass
            info = model.train_on_batch(batch=input_batch, epoch=epoch, validate=False)

            # record loss
            step_log = model.log_info(info)
            losses.append(step_log["Loss"])

        # do anything model needs to after finishing epoch
        model.on_epoch_end(epoch)

        print("Train Epoch {}: Loss {}".format(epoch, np.mean(losses)))


## 5. Run policy training

Using the model and the training loop defined above. Note that this simple training loop does not save checkpoint. For model checkpointing, take a look at the full-feature [training loop](https://github.com/ARISE-Initiative/robomimic/blob/master/robomimic/scripts/train.py#L290) and the [documentation](https://robomimic.github.io/docs/tutorials/viewing_results.html)

In [9]:
# get dataset loader
data_loader = get_data_loader(dataset_path=dataset_path)

# run training loop
run_train_loop(model=model, data_loader=data_loader, num_epochs=50, gradient_steps_per_epoch=100)

SequenceDataset: loading dataset into memory...
100%|██████████| 200/200 [00:00<00:00, 208.16it/s]
SequenceDataset: caching get_item calls...
100%|██████████| 9666/9666 [00:03<00:00, 2583.00it/s]

============= Created Dataset =============
SequenceDataset (
	path=/content/robomimic_data/low_dim_v15.hdf5
	obs_keys=('robot0_eef_pos', 'robot0_eef_quat', 'robot0_gripper_qpos', 'object')
	seq_length=10
	filter_key=none
	frame_stack=1
	pad_seq_length=True
	pad_frame_stack=True
	goal_mode=none
	cache_mode=all
	num_demos=200
	num_sequences=9666
)

Train Epoch 1: Loss 0.15334104910492896
Train Epoch 2: Loss 0.10758635729551315
Train Epoch 3: Loss 0.07848162800073624
Train Epoch 4: Loss 0.06477158192545175
Train Epoch 5: Loss 0.05516338497400284
Train Epoch 6: Loss 0.0492780664563179
Train Epoch 7: Loss 0.04519758947193622
Train Epoch 8: Loss 0.04282913943752646
Train Epoch 9: Loss 0.040431210026144984
Train Epoch 10: Loss 0.038654641453176736
Train Epoch 11: Loss 0.03748038278892636
Train Epoc

## 6. Evaluate and visualize trained policy

Here we execute the trained policy `model` in a simulated environment and play the rollout video.

In [13]:
# create simulation environment

import robomimic.utils.env_utils as EnvUtils

env_meta = FileUtils.get_env_metadata_from_dataset(dataset_path)

env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta,
    env_name=env_meta["env_name"],
    render=False,
    render_offscreen=True,
    use_image_obs=False,
)

Created environment with name Lift
Action size is 7


In [14]:
from robomimic.algo import RolloutPolicy
from robomimic.utils.train_utils import run_rollout
import imageio

# create a thin wrapper around the model to interact with the environment
policy = RolloutPolicy(model)

# create a video writer
video_path = "rollout.mp4"
video_writer = imageio.get_writer(video_path, fps=20)

# run rollout
rollout_log = run_rollout(
    policy=policy,
    env=env,
    horizon=200,
    video_writer=video_writer,
    render=False
)

video_writer.close()
# print rollout results
print(rollout_log)

{'Return': 96.0, 'Horizon': 200, 'Success_Rate': 1.0}


In [15]:
# visualize rollout video

from IPython.display import HTML
from base64 import b64encode

mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

## What's next?

Now that you understand the basic components of robomimic, it's time to delve deeper into each component by reading up the [documentation site](https://robomimic.github.io/docs/introduction/overview.html). Robomimic offers a rich set of utilities for model building, training loop management, model checkpointing, visualization, hyper-parameter sweeping, and logging. You can get to more about each component by following the provided the examples and tutorials.

